In [2]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.eval_measures import aic, bic
from sklearn.metrics import r2_score
import patsy

In [2]:
df_90s = pd.read_csv(r'D:\vr-pipeline\data\results\22052025_segment_data_t123_90s_1s_ketone.csv')
df_30s = pd.read_csv(r'D:\vr-pipeline\data\results\22052025_segment_data_t123_30s_30s_ketone.csv')

# Example: Load datasets
df_90s['segmentation'] = '90s_1s'
df_30s['segmentation'] = '30s_30s'   
df_all = pd.concat([df_90s, df_30s], ignore_index=True)

# 90s analysis

## empty room

In [3]:
df_90_empty_room_pca=pd.read_csv(r'D:\vr-pipeline\data\pca\combined_90_empty_room_pca.csv')

In [4]:
scene_map = {7: 'empty_room', 11: 'elevated_platform', 15: 'dark_maze'}
df_all['scene'] = df_all['trial'].map(scene_map)

In [5]:
df_90_empty_room_pca.loc[df_90_empty_room_pca['drink'] == 'placebo', 'drink'] = -0.5
df_90_empty_room_pca.loc[df_90_empty_room_pca['drink'] == 'ketone', 'drink'] = 0.5

In [6]:
df_all['unsafe_time_spent']= df_all['unsafe_time_spent'] + df_all['outside_time_spent']

In [7]:
import numpy as np
features=['PC1','PC2','PC3']
for feature in features:
    model = smf.mixedlm(f"{feature} ~ drink + C(session)",
                        data=df_90_empty_room_pca,
                        groups=df_90_empty_room_pca["participant_id"])
    result = model.fit()
    print(result.summary())
    
    # Calculate variance components for R-squared
    fixed_effects_variance = np.var(result.fittedvalues)  
    random_effects_variance = result.cov_re.iloc[0, 0] 
    residual_variance = result.scale  
    
    # Calculate Marginal and Conditional R^2
    R2_m = fixed_effects_variance / (fixed_effects_variance + random_effects_variance + residual_variance)
    R2_c = (fixed_effects_variance + random_effects_variance) / (fixed_effects_variance + random_effects_variance + residual_variance)
    
    print(f"Marginal R² = {R2_m:.4f}")
    print(f"Conditional R² = {R2_c:.4f}\n")

          Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  PC1     
No. Observations:   38       Method:              REML    
No. Groups:         19       Scale:               0.8404  
Min. group size:    2        Log-Likelihood:      -69.0594
Max. group size:    2        Converged:           Yes     
Mean group size:    2.0                                   
----------------------------------------------------------
                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept       -0.116    0.461 -0.253 0.800 -1.020  0.787
drink[T.0.5]     0.019    0.298  0.063 0.950 -0.565  0.603
C(session)[T.2]  0.214    0.298  0.719 0.472 -0.370  0.798
Group Var        2.815    1.688                           

Marginal R² = 0.3895
Conditional R² = 0.8596

          Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  PC2     
No. Observations:   38       Me

In [6]:
df_90_empty_room_pca=pd.read_csv(r'D:\vr-pipeline\data\pca\combined_90_empty_room_pca.csv')

df_90_empty_room_pca.loc[df_90_empty_room_pca['drink'] == 'placebo', 'drink'] = -0.5
df_90_empty_room_pca.loc[df_90_empty_room_pca['drink'] == 'ketone', 'drink'] = 0.5
df_90_empty_room_pca.loc[df_90_empty_room_pca['session'] == 1, 'session'] = -0.5
df_90_empty_room_pca.loc[df_90_empty_room_pca['session'] == 2, 'session'] = 0.5


features=['PC1','PC2','PC3']
for feature in features:
    model = smf.mixedlm(f"{feature} ~ drink + C(session)",
                        data=df_90_empty_room_pca,
                        groups=df_90_empty_room_pca["participant_id"])
    result = model.fit()
    print(result.summary())
    

           Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   PC1     
No. Observations:   38        Method:               REML    
No. Groups:         19        Scale:                0.8404  
Min. group size:    2         Log-Likelihood:       -69.0594
Max. group size:    2         Converged:            Yes     
Mean group size:    2.0                                     
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         -0.116    0.461 -0.253 0.800 -1.020  0.787
drink[T.0.5]       0.019    0.298  0.063 0.950 -0.565  0.603
C(session)[T.0.5]  0.214    0.298  0.719 0.472 -0.370  0.798
Group Var          2.815    1.688                           

           Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   PC2     
No. Observations:   38        Method:          

C:\Users\lal\AppData\Local\Temp\ipykernel_20544\291832775.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_90_empty_room_pca.loc[df_90_empty_room_pca['session'] == 1, 'session'] = -0.5


In [5]:
features=['PC1','PC2','PC3']
for feature in features:
    model = smf.mixedlm(f"{feature} ~ drink + C(session)",
                        data=df_90_empty_room_pca,
                        groups=df_90_empty_room_pca["participant_id"])
    result = model.fit()
    print(result.summary())

NameError: name 'df_90_empty_room_pca' is not defined

In [10]:
from scipy.stats import ttest_rel

# Only valid if drink is not a confounder
session_1 = df_90_empty_room_pca.loc[df_90_empty_room_pca['session'] == 1].sort_values('participant_id')['PC1']

session_2 = df_90_empty_room_pca[df_90_empty_room_pca['session'] == 2].sort_values('participant_id')['PC1']

ttest_rel(session_1.values, session_2.values)


TtestResult(statistic=np.float64(-0.744482232352145), pvalue=np.float64(0.4661940503431904), df=np.int64(18))

In [11]:
# import os
# import polars as pl
# from pymer4.models import lmer

# # Load the CSV file as a Polars DataFrame
# csv_path = r"D:\vr-pipeline\data\pca\combined_90_empty_room_pca.csv"
# df_90_empty_room_pca = pl.read_csv(csv_path)
# # Replace categorical values with numeric in Polars
# polars_df = df_90_empty_room_pca.with_columns(
#     pl.when(pl.col('drink') == 'placebo')
#     .then(-0.5)
#     .when(pl.col('drink') == 'ketone')
#     .then(0.5)
#     .alias('drink')
# )

# features = ['PC1', 'PC2', 'PC3']

# for feature in features:
#     model = lmer(f"{feature} ~ drink + C(session) + (1 | participant_id)", data=polars_df)
#     results=model.fit(summary=True)
#     print(f"\n=== Results for {feature} ===")
#     print(results)
#     #print(result)

In [12]:
# model.summary()

In [13]:
'''
#df 90 empty room PCA multivariate regression
from sklearn.linear_model import LinearRegression
import pandas as pd
import statsmodels.api as sm

results = []
i = 0

features = ['PC1', 'PC2', 'PC3']

# x and y for regression
X = df_90_empty_room_pca.drop(columns=['participant_id', 'PC1', 'PC2', 'PC3'], axis=1)
y = df_90_empty_room_pca[['PC1', 'PC2', 'PC3']]

model = LinearRegression()
model.fit(X, y)

r2 = model.score(X, y)
coefs = dict(zip(X.columns, model.coef_))
coefs['intercept'] = model.intercept_
display(coefs)
'''

"\n#df 90 empty room PCA multivariate regression\nfrom sklearn.linear_model import LinearRegression\nimport pandas as pd\nimport statsmodels.api as sm\n\nresults = []\ni = 0\n\nfeatures = ['PC1', 'PC2', 'PC3']\n\n# x and y for regression\nX = df_90_empty_room_pca.drop(columns=['participant_id', 'PC1', 'PC2', 'PC3'], axis=1)\ny = df_90_empty_room_pca[['PC1', 'PC2', 'PC3']]\n\nmodel = LinearRegression()\nmodel.fit(X, y)\n\nr2 = model.score(X, y)\ncoefs = dict(zip(X.columns, model.coef_))\ncoefs['intercept'] = model.intercept_\ndisplay(coefs)\n"

In [14]:
'''
import pandas as pd
import statsmodels.api as sm

# Convert 'drink' to numeric
X['drink'] = pd.to_numeric(X['drink'], errors='coerce')


# Drop rows with missing values
X = X.dropna()
y = y.dropna()

# Add a constant to X for the intercept
X_with_constant = sm.add_constant(X)

# Fit OLS for each dependent variable separately
for col in y.columns:
    print(f"Regression results for {col}:")
    
    # Fit the model
    model = sm.OLS(y[col], X_with_constant)
    results = model.fit()
    
    # Display the summary
    print(results.summary())
    print("\n")
'''

'\nimport pandas as pd\nimport statsmodels.api as sm\n\n# Convert \'drink\' to numeric\nX[\'drink\'] = pd.to_numeric(X[\'drink\'], errors=\'coerce\')\n\n\n# Drop rows with missing values\nX = X.dropna()\ny = y.dropna()\n\n# Add a constant to X for the intercept\nX_with_constant = sm.add_constant(X)\n\n# Fit OLS for each dependent variable separately\nfor col in y.columns:\n    print(f"Regression results for {col}:")\n\n    # Fit the model\n    model = sm.OLS(y[col], X_with_constant)\n    results = model.fit()\n\n    # Display the summary\n    print(results.summary())\n    print("\n")\n'

## elevated platform

In [7]:
df_90_elevated_platform_pca=pd.read_csv(r'D:\vr-pipeline\data\pca\combined_90_elevated_platform_pca.csv')

In [8]:
df_90_elevated_platform_pca.loc[df_90_elevated_platform_pca['drink'] == 'placebo', 'drink'] = -0.5
df_90_elevated_platform_pca.loc[df_90_elevated_platform_pca['drink'] == 'ketone', 'drink'] = 0.5
df_90_elevated_platform_pca.loc[df_90_elevated_platform_pca['session'] == 1, 'session'] = -0.5
df_90_elevated_platform_pca.loc[df_90_elevated_platform_pca['session'] == 2, 'session'] = 0.5

C:\Users\lal\AppData\Local\Temp\ipykernel_20544\2866770987.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_90_elevated_platform_pca.loc[df_90_elevated_platform_pca['session'] == 1, 'session'] = -0.5


In [9]:
features=['PC1','PC2','PC3']
for feature in features:
    model = smf.mixedlm(f"{feature} ~ drink + C(session)",
                        data=df_90_elevated_platform_pca,
                        groups=df_90_elevated_platform_pca["participant_id"])
    result = model.fit()
    print(result.summary())

           Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   PC1     
No. Observations:   38        Method:               REML    
No. Groups:         19        Scale:                1.0555  
Min. group size:    2         Log-Likelihood:       -74.7683
Max. group size:    2         Converged:            Yes     
Mean group size:    2.0                                     
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         -0.246    0.558 -0.441 0.659 -1.341  0.848
drink[T.0.5]      -0.507    0.334 -1.520 0.129 -1.161  0.147
C(session)[T.0.5]  1.000    0.334  2.995 0.003  0.346  1.654
Group Var          4.392    2.290                           

           Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   PC2     
No. Observations:   38        Method:          

In [10]:
features=['PC1','PC2','PC3']
for feature in features:
    model = smf.mixedlm(f"{feature} ~ drink + C(session)",
                        data=df_90_elevated_platform_pca,
                        groups=df_90_elevated_platform_pca["participant_id"])
    result = model.fit()
    print(result.summary())

           Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   PC1     
No. Observations:   38        Method:               REML    
No. Groups:         19        Scale:                1.0555  
Min. group size:    2         Log-Likelihood:       -74.7683
Max. group size:    2         Converged:            Yes     
Mean group size:    2.0                                     
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         -0.246    0.558 -0.441 0.659 -1.341  0.848
drink[T.0.5]      -0.507    0.334 -1.520 0.129 -1.161  0.147
C(session)[T.0.5]  1.000    0.334  2.995 0.003  0.346  1.654
Group Var          4.392    2.290                           

           Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   PC2     
No. Observations:   38        Method:          

In [21]:
features=['PC1','PC2','PC3']
for feature in features:
    model = smf.mixedlm(f"{feature} ~ C(session) + drink",
                        data=df_90_elevated_platform_pca,
                        groups=df_90_elevated_platform_pca["participant_id"])
    result = model.fit()
    print(result.summary())

          Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  PC1     
No. Observations:   38       Method:              REML    
No. Groups:         19       Scale:               1.0555  
Min. group size:    2        Log-Likelihood:      -74.7683
Max. group size:    2        Converged:           Yes     
Mean group size:    2.0                                   
----------------------------------------------------------
                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept       -0.246    0.558 -0.441 0.659 -1.341  0.848
C(session)[T.2]  1.000    0.334  2.995 0.003  0.346  1.654
drink[T.0.5]    -0.507    0.334 -1.520 0.129 -1.161  0.147
Group Var        4.392    2.290                           

          Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  PC2     
No. Observations:   38       Method:              REML    
No. Groups:       

In [23]:
features=['PC1','PC2','PC3']
for feature in features:
    model = smf.mixedlm(f"{feature} ~ drink * C(session)",
                        data=df_90_elevated_platform_pca,
                        groups=df_90_elevated_platform_pca["participant_id"])
    result = model.fit()
    print(result.summary())

                 Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      PC1     
No. Observations:        38           Method:                  REML    
No. Groups:              19           Scale:                   1.0555  
Min. group size:         2            Log-Likelihood:          -73.1108
Max. group size:         2            Converged:               Yes     
Mean group size:         2.0                                           
-----------------------------------------------------------------------
                             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------
Intercept                    -0.168    0.757 -0.222 0.824 -1.652  1.315
drink[T.0.5]                 -0.672    1.100 -0.611 0.541 -2.827  1.484
C(session)[T.2]               0.835    1.100  0.760 0.448 -1.320  2.991
drink[T.0.5]:C(session)[T.2]  0.329    2.096  0.157 0.875 -3.779  4.437
Group Var

## dark maze

In [12]:
df_90_dark_maze_pca=pd.read_csv(r'D:\vr-pipeline\data\pca\combined_90_dark_maze_pca.csv')

In [13]:
df_90_dark_maze_pca.loc[df_90_dark_maze_pca['drink'] == 'placebo', 'drink'] = -0.5
df_90_dark_maze_pca.loc[df_90_dark_maze_pca['drink'] == 'ketone', 'drink'] = 0.5
df_90_dark_maze_pca.loc[df_90_dark_maze_pca['session'] == 1, 'session'] = -0.5
df_90_dark_maze_pca.loc[df_90_dark_maze_pca['session'] == 2, 'session'] = 0.5

C:\Users\lal\AppData\Local\Temp\ipykernel_20544\3526809510.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_90_dark_maze_pca.loc[df_90_dark_maze_pca['session'] == 1, 'session'] = -0.5


In [14]:
features=['PC1','PC2','PC3']
for feature in features:
    model = smf.mixedlm(f"{feature} ~ drink + C(session)",
                        data=df_90_dark_maze_pca,
                        groups=df_90_dark_maze_pca["participant_id"])
    result = model.fit()
    print(result.summary())

           Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   PC1     
No. Observations:   38        Method:               REML    
No. Groups:         19        Scale:                1.4356  
Min. group size:    2         Log-Likelihood:       -71.0069
Max. group size:    2         Converged:            Yes     
Mean group size:    2.0                                     
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         -0.580    0.446 -1.299 0.194 -1.455  0.295
drink[T.0.5]       0.192    0.389  0.493 0.622 -0.571  0.955
C(session)[T.0.5]  0.968    0.389  2.487 0.013  0.205  1.731
Group Var          1.705    0.967                           

           Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   PC2     
No. Observations:   38        Method:          

In [27]:
features=['PC1','PC2','PC3']
for feature in features:
    model = smf.mixedlm(f"{feature} ~ drink * C(session)",
                        data=df_90_dark_maze_pca,
                        groups=df_90_dark_maze_pca["participant_id"])
    result = model.fit()
    print(result.summary())

                 Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      PC1     
No. Observations:        38           Method:                  REML    
No. Groups:              19           Scale:                   1.4356  
Min. group size:         2            Log-Likelihood:          -68.5423
Max. group size:         2            Converged:               Yes     
Mean group size:         2.0                                           
-----------------------------------------------------------------------
                             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------
Intercept                    -1.096    0.543 -2.017 0.044 -2.161 -0.031
drink[T.0.5]                  1.282    0.789  1.623 0.104 -0.266  2.829
C(session)[T.2]               2.058    0.789  2.606 0.009  0.510  3.605
drink[T.0.5]:C(session)[T.2] -2.179    1.374 -1.587 0.113 -4.871  0.513
Group Var

# 30s analysis

In [23]:
df_30_empty_room_pca = pd.read_csv(r'D:\vr-pipeline\data\pca\combined_30_empty_room_pca.csv')

df_30_empty_room_pca.loc[df_30_empty_room_pca['drink'] == 'placebo', 'drink'] = -0.5
df_30_empty_room_pca.loc[df_30_empty_room_pca['drink'] == 'ketone', 'drink'] = 0.5
df_30_empty_room_pca['session'] = df_30_empty_room_pca['session'].astype(float)
df_30_empty_room_pca.loc[df_30_empty_room_pca['session'] == 1, 'session'] = -0.5
df_30_empty_room_pca.loc[df_30_empty_room_pca['session'] == 2, 'session'] = 0.5

features=['PC1','PC2','PC3']

for feature in features:
    model = smf.mixedlm(f"{feature} ~ (C(segment) + C(session) + drink) ** 2 ",
                        data=df_30_empty_room_pca,
                        groups=df_30_empty_room_pca["participant_id"])
    result = model.fit()
    print(result.summary())

                   Mixed Linear Model Regression Results
Model:                   MixedLM        Dependent Variable:        PC1      
No. Observations:        114            Method:                    REML     
No. Groups:              19             Scale:                     1.5041   
Min. group size:         6              Log-Likelihood:            -201.7111
Max. group size:         6              Converged:                 Yes      
Mean group size:         6.0                                                
----------------------------------------------------------------------------
                                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------
Intercept                          0.116    0.597  0.194 0.846 -1.055  1.287
C(segment)[T.1]                    0.220    0.479  0.460 0.646 -0.719  1.159
C(segment)[T.2]                   -0.717    0.479 -1.496 0.135 -1.656  0.222
C(session)[T.0.5]  

In [7]:
df_30_empty_room_pca['segment'].value_counts()

segment
0    38
1    38
2    38
Name: count, dtype: int64

In [24]:
df_30_elevated_platform_pca = pd.read_csv(r'D:\vr-pipeline\data\pca\combined_30_elevated_platform_pca.csv')

df_30_elevated_platform_pca.loc[df_30_elevated_platform_pca['drink'] == 'placebo', 'drink'] = -0.5
df_30_elevated_platform_pca.loc[df_30_elevated_platform_pca['drink'] == 'ketone', 'drink'] = 0.5
df_30_elevated_platform_pca['session'] = df_30_elevated_platform_pca['session'].astype(float)
df_30_elevated_platform_pca.loc[df_30_elevated_platform_pca['session'] == 1, 'session'] = -0.5
df_30_elevated_platform_pca.loc[df_30_elevated_platform_pca['session'] == 2, 'session'] = 0.5

for feature in features:
    model = smf.mixedlm(f"{feature} ~ (C(segment) + C(session) + drink) ** 2",
                        data=df_30_elevated_platform_pca,
                        groups=df_30_elevated_platform_pca["participant_id"])
    result = model.fit()
    print(result.summary())

                   Mixed Linear Model Regression Results
Model:                   MixedLM        Dependent Variable:        PC1      
No. Observations:        114            Method:                    REML     
No. Groups:              19             Scale:                     1.4381   
Min. group size:         6              Log-Likelihood:            -203.2234
Max. group size:         6              Converged:                 Yes      
Mean group size:         6.0                                                
----------------------------------------------------------------------------
                                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------
Intercept                          0.320    0.694  0.461 0.645 -1.039  1.679
C(segment)[T.1]                   -0.583    0.469 -1.244 0.214 -1.501  0.336
C(segment)[T.2]                   -0.906    0.469 -1.934 0.053 -1.825  0.012
C(session)[T.0.5]  

In [ ]:
import statsmodels.formula.api as smf
import pandas as pd
features=['PC1','PC2','PC3']

df_30_dark_maze_pca = pd.read_csv(r'D:\vr-pipeline\data\pca\combined_30_dark_maze_pca.csv')



df_30_dark_maze_pca.loc[df_30_dark_maze_pca['drink'] == 'placebo', 'drink'] = -0.5
df_30_dark_maze_pca.loc[df_30_dark_maze_pca['drink'] == 'ketone', 'drink'] = 0.5

for feature in features:
    model = smf.mixedlm(f"{feature} ~ (C(segment) + drink + C(session)) **2",
                        data=df_30_dark_maze_pca,
                        groups=df_30_dark_maze_pca["participant_id"])
    result = model.fit()
    print(result.summary())

                  Mixed Linear Model Regression Results
Model:                   MixedLM       Dependent Variable:       PC1      
No. Observations:        114           Method:                   REML     
No. Groups:              19            Scale:                    1.3795   
Min. group size:         6             Log-Likelihood:           -194.1980
Max. group size:         6             Converged:                Yes      
Mean group size:         6.0                                              
--------------------------------------------------------------------------
                                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------
Intercept                       -1.983    0.507 -3.908 0.000 -2.977 -0.988
C(segment)[T.1]                  1.812    0.459  3.949 0.000  0.913  2.711
C(segment)[T.2]                  1.968    0.459  4.289 0.000  1.069  2.867
drink[T.0.5]                     0.849    0.

In [20]:
import statsmodels.formula.api as smf
import pandas as pd
features=['PC1','PC2','PC3']

df_30_dark_maze_pca = pd.read_csv(r'D:\vr-pipeline\data\pca\combined_30_dark_maze_pca.csv')



df_30_dark_maze_pca.loc[df_30_dark_maze_pca['drink'] == 'placebo', 'drink'] = -0.5
df_30_dark_maze_pca.loc[df_30_dark_maze_pca['drink'] == 'ketone', 'drink'] = 0.5

df_30_dark_maze_pca['session'] = df_30_dark_maze_pca['session'].astype(float)
df_30_dark_maze_pca.loc[df_30_dark_maze_pca['session'] == 1, 'session'] = -0.5
df_30_dark_maze_pca.loc[df_30_dark_maze_pca['session'] == 2, 'session'] = 0.5

for feature in features:
    model = smf.mixedlm(f"{feature} ~ C(segment) * drink * C(session) ",
                        data=df_30_dark_maze_pca,
                        groups=df_30_dark_maze_pca["participant_id"])
    result = model.fit()
    print(result.summary())

                          Mixed Linear Model Regression Results
Model:                        MixedLM            Dependent Variable:            PC1      
No. Observations:             114                Method:                        REML     
No. Groups:                   19                 Scale:                         1.4050   
Min. group size:              6                  Log-Likelihood:                -192.1304
Max. group size:              6                  Converged:                     Yes      
Mean group size:              6.0                                                        
-----------------------------------------------------------------------------------------
                                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      -2.020    0.530 -3.808 0.000 -3.059 -0.980
C(segment)[T.1]                     

In [4]:
import statsmodels.formula.api as smf
import pandas as pd
features=['PC1','PC2','PC3']

df_30_dark_maze_pca = pd.read_csv(r'D:\vr-pipeline\data\pca\combined_30_dark_maze_pca.csv')



df_30_dark_maze_pca.loc[df_30_dark_maze_pca['drink'] == 'placebo', 'drink'] = -0.5
df_30_dark_maze_pca.loc[df_30_dark_maze_pca['drink'] == 'ketone', 'drink'] = 0.5
for feature in features:
    model = smf.mixedlm(f"{feature} ~ (drink + C(segment) + C(session))**3",
                        data=df_30_dark_maze_pca,
                        groups=df_30_dark_maze_pca["participant_id"])
    result = model.fit()
    print(result.summary())

                         Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           PC1      
No. Observations:             114               Method:                       REML     
No. Groups:                   19                Scale:                        1.4050   
Min. group size:              6                 Log-Likelihood:               -192.1304
Max. group size:              6                 Converged:                    Yes      
Mean group size:              6.0                                                      
---------------------------------------------------------------------------------------
                                             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------
Intercept                                    -2.020    0.530 -3.808 0.000 -3.059 -0.980
drink[T.0.5]                                  0.927    0.

In [34]:
for feature in features:
    model = smf.mixedlm(f"{feature} ~ C(segment) + drink * C(session)",
                        data=df_30_elevated_platform_pca,
                        groups=df_30_elevated_platform_pca["participant_id"])
    result = model.fit()
    print(result.summary())

                 Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      PC1      
No. Observations:       114          Method:                  REML     
No. Groups:             19           Scale:                   1.4664   
Min. group size:        6            Log-Likelihood:          -207.1124
Max. group size:        6            Converged:               Yes      
Mean group size:        6.0                                            
-----------------------------------------------------------------------
                             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------
Intercept                     0.305    0.659  0.463 0.643 -0.986  1.596
C(segment)[T.1]              -0.542    0.278 -1.951 0.051 -1.086  0.003
C(segment)[T.2]              -0.903    0.278 -3.249 0.001 -1.447 -0.358
drink[T.0.5]                 -0.502    0.928 -0.540 0.589 -2.321  1.318
C(session